In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter
from seaborn import set_style
import kagglehub
set_style("whitegrid")

In [2]:
path = kagglehub.dataset_download("cryptrader/huge-japanese-stock-market-dataset-all-in-one")

print("Path to dataset files:", path)

100%|████████████████████████████████████████| 218M/218M [01:33<00:00, 2.45MB/s]

Extracting model files...


Path to dataset files: /home/hava/.cache/kagglehub/datasets/cryptrader/huge-japanese-stock-market-dataset-all-in-one/versions/1


In [28]:
prices_file = path + '/all_japanese_stocks.csv'
df_prices = pd.read_csv(prices_file)
df_japan_long = df_prices.assign(ticker=df_prices['Unnamed: 0'])
df_japan_long

,Unnamed: 0,Date,High,Low,Open,Close,Volume,Adj Close,ticker
0,1301,2001-01-01,1400.0,1400.0,1400.0,1400.0,0.0,1003.504517,1301
1,1301,2001-01-02,1400.0,1400.0,1400.0,1400.0,0.0,1003.504517,1301
2,1301,2001-01-03,1400.0,1400.0,1400.0,1400.0,0.0,1003.504517,1301
3,1301,2001-01-04,1420.0,1370.0,1420.0,1390.0,19300.0,996.336853,1301
4,1301,2001-01-05,1400.0,1330.0,1390.0,1330.0,19700.0,953.329529,1301
...,...,...,...,...,...,...,...,...,...
14356282,9997,2021-02-03,1099.0,1034.0,1097.0,1042.0,702200.0,1042.000000,9997
14356283,9997,2021-02-04,1061.0,1016.0,1040.0,1025.0,416700.0,1025.000000,9997
14356284,9997,2021-02-05,1051.0,1027.0,1033.0,1048.0,256000.0,1048.000000,9997
14356285,9997,2021-02-08,1060.0,1045.0,1053.0,1054.0,221100.0,1054.000000,9997


In [39]:
df_japan_long['ticker'].unique().astype(str)[:10]
#df_japan[df_japan.ticker=='1301']
tkr_list = df_japan_long['ticker'].unique()[:20]
print(tkr_list)
df_japan = df_japan_long[df_japan_long.ticker.isin(tkr_list)]
df_japan

[1301 1305 1306 1308 1309 1311 1312 1313 1319 1320 1321 1322 1323 1324
 1325 1326 1327 1328 1329 1330]


,Unnamed: 0,Date,High,Low,Open,Close,Volume,Adj Close,ticker
0,1301,2001-01-01,1400.0,1400.0,1400.0,1400.0,0.0,1003.504517,1301
1,1301,2001-01-02,1400.0,1400.0,1400.0,1400.0,0.0,1003.504517,1301
2,1301,2001-01-03,1400.0,1400.0,1400.0,1400.0,0.0,1003.504517,1301
3,1301,2001-01-04,1420.0,1370.0,1420.0,1390.0,19300.0,996.336853,1301
4,1301,2001-01-05,1400.0,1330.0,1390.0,1330.0,19700.0,953.329529,1301
...,...,...,...,...,...,...,...,...,...
63773,1330,2021-02-03,29620.0,29360.0,29400.0,29590.0,26800.0,29590.000000,1330
63774,1330,2021-02-04,29540.0,29260.0,29510.0,29290.0,34520.0,29290.000000,1330
63775,1330,2021-02-05,29710.0,29480.0,29580.0,29700.0,45460.0,29700.000000,1330
63776,1330,2021-02-08,30350.0,29770.0,29790.0,30300.0,77630.0,30300.000000,1330


In [40]:
df_japan['Date'] = pd.to_datetime(df_japan['Date'])
dst_dates_df = pd.read_csv('DaylightSavingsTimeChangeDates_1971-2024.csv')
dst_dates_spring = pd.to_datetime(dst_dates_df.Date+" "+dst_dates_df.Year.astype(str))
dst_dates_fall = pd.to_datetime(dst_dates_df['Date.1']+" "+dst_dates_df.Year.astype(str))
dst_dates = pd.concat([dst_dates_fall,dst_dates_spring])
dst_mon = dst_dates+pd.DateOffset(1)
dst_fri = dst_dates+pd.DateOffset(-2)
dst_fall_mon = dst_dates_fall+pd.DateOffset(1)
dst_fall_fri = dst_dates_fall+pd.DateOffset(-2)
dst_spring_mon = dst_dates_spring+pd.DateOffset(1)
dst_spring_fri = dst_dates_spring+pd.DateOffset(-2)

/tmp/ipykernel_27077/3779876777.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_japan['Date'] = pd.to_datetime(df_japan['Date'])


In [41]:
japan_dst_mon_fall = df_japan[df_japan.Date.isin(dst_fall_mon)]
japan_dst_mon_fall = japan_dst_mon_fall.assign(Season='Fall')
japan_dst_mon_spring = df_japan[df_japan.Date.isin(dst_spring_mon)]
japan_dst_mon_spring = japan_dst_mon_spring.assign(Season='Spring')
japan_dst_mon = pd.concat([japan_dst_mon_fall,japan_dst_mon_spring])
japan_dst_fri = df_japan[df_japan.Date.isin(dst_fri)]

for tkr in tkr_list:
    mon_tkr = japan_dst_mon[japan_dst_mon.ticker==tkr]
    fri_tkr = japan_dst_fri[japan_dst_fri.ticker==tkr]
    japan_dst_mon = japan_dst_mon.drop(mon_tkr[mon_tkr.Date.isin(fri_tkr.Date+pd.DateOffset(3))==False].index)
    japan_dst_fri = japan_dst_fri.drop(fri_tkr[fri_tkr.Date.isin(mon_tkr.Date+pd.DateOffset(-3))==False].index)
    #finance_dst_mon = finance_dst_mon.drop(finance_dst_mon[(finance_dst_mon.ticker.astype(str)==tkr  finance_dst_mon['date'].isin(finance_dst_fri[finance_dst_fri.ticker.astype(str)==tkr].date+pd.DateOffset(3))==False)].index)
    #finance_dst_fri = finance_dst_fri.drop(finance_dst_fri[(finance_dst_fri['ticker'==tkr] & finance_dst_fri['date'].isin(finance_dst_mon['ticker'==tkr].date+pd.DateOffset(-3))==False)].index)
print(len(japan_dst_mon))
print(len(japan_dst_fri))

443
443


In [42]:
japan_dst_fri = japan_dst_fri.reset_index()
japan_dst_mon = japan_dst_mon.reset_index()

In [43]:
japan_dst = pd.DataFrame()
japan_dst['ticker'] = japan_dst_mon['ticker']
japan_dst['mon_date'] = japan_dst_mon['Date']
japan_dst['mon_open'] = japan_dst_mon['Open']
japan_dst['mon_close'] = japan_dst_mon['Close']
japan_dst['mon_adj_close'] = japan_dst_mon['Adj Close']
japan_dst['mon_low'] = japan_dst_mon['Low']
japan_dst['mon_high'] = japan_dst_mon['High']
japan_dst['mon_volume'] = japan_dst_mon['Volume']
japan_dst['fri_date'] = japan_dst_fri['Date']
japan_dst['fri_open'] = japan_dst_fri['Open']
japan_dst['fri_close'] = japan_dst_fri['Close']
japan_dst['fri_adj_close'] = japan_dst_fri['Adj Close']
japan_dst['fri_low'] = japan_dst_fri['Low']
japan_dst['fri_high'] = japan_dst_fri['High']
japan_dst['fri_volume'] = japan_dst_fri['Volume']
japan_dst['Season'] = japan_dst_mon['Season']
japan_dst

,ticker,mon_date,mon_open,mon_close,mon_adj_close,mon_low,mon_high,mon_volume,fri_date,fri_open,fri_close,fri_adj_close,fri_low,fri_high,fri_volume,Season
0,1301,2001-10-29,1410.0,1410.0,1030.360840,1400.0,1420.0,10000.0,2001-03-30,1520.0,1480.0,1081.513794,1480.0,1540.0,22500.0,Fall
1,1301,2002-10-28,1000.0,1000.0,746.300293,980.0,1000.0,15600.0,2001-10-26,1420.0,1400.0,1023.053223,1400.0,1440.0,25800.0,Fall
2,1301,2003-10-27,1670.0,1690.0,1301.933105,1660.0,1720.0,52600.0,2002-04-05,1450.0,1450.0,1082.135620,1430.0,1460.0,10300.0,Fall
3,1301,2004-11-01,1990.0,2000.0,1572.192993,1980.0,2020.0,42100.0,2002-10-25,980.0,1000.0,746.300293,980.0,1000.0,30400.0,Fall
4,1301,2005-10-31,2780.0,2790.0,2193.208984,2770.0,2810.0,104600.0,2003-04-04,1210.0,1240.0,955.264465,1210.0,1250.0,26700.0,Fall
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
438,1330,2016-03-14,17690.0,17720.0,17720.000000,17640.0,17800.0,110200.0,2018-03-09,22250.0,22130.0,22130.000000,22030.0,22570.0,188620.0,Spring
439,1330,2017-03-13,20100.0,20190.0,20190.000000,20090.0,20230.0,45330.0,2018-11-02,22540.0,22960.0,22960.000000,22500.0,23010.0,109130.0,Spring
440,1330,2018-03-12,22630.0,22540.0,22540.000000,22380.0,22670.0,135590.0,2019-03-08,22040.0,21740.0,21740.000000,21710.0,22050.0,62960.0,Spring
441,1330,2019-03-11,21790.0,21820.0,21820.000000,21650.0,21860.0,32580.0,2020-03-06,21700.0,21430.0,21430.000000,21320.0,21770.0,139910.0,Spring


In [45]:
japan_mon = df_japan[df_japan.Date.dt.day_name()=='Monday']
japan_mon = japan_mon.drop(japan_mon[japan_mon.Date.isin(dst_mon)].index)
japan_fri = df_japan[df_japan.Date.dt.day_name()=='Friday']
japan_fri = japan_fri.drop(japan_fri[japan_fri.Date.isin(dst_fri)].index)
for tkr in tkr_list:
    mon_tkr = japan_mon[japan_mon.ticker==tkr]
    fri_tkr = japan_fri[japan_fri.ticker==tkr]
    japan_mon = japan_mon.drop(mon_tkr[mon_tkr.Date.isin(fri_tkr.Date+pd.DateOffset(3))==False].index)
    japan_fri = japan_fri.drop(fri_tkr[fri_tkr.Date.isin(mon_tkr.Date+pd.DateOffset(-3))==False].index)
print(len(japan_mon))
print(len(japan_fri))

11102
11102


In [46]:
japan_mon = japan_mon.reset_index()
japan_fri = japan_fri.reset_index()

In [47]:
japan_wkd = pd.DataFrame()
japan_wkd['ticker'] = japan_mon['ticker']
japan_wkd['mon_date'] = japan_mon['Date']
japan_wkd['mon_open'] = japan_mon['Open']
japan_wkd['mon_close'] = japan_mon['Close']
japan_wkd['mon_adj_close'] = japan_mon['Adj Close']
japan_wkd['mon_low'] = japan_mon['Low']
japan_wkd['mon_high'] = japan_mon['High']
japan_wkd['mon_volume'] = japan_mon['Volume']
japan_wkd['fri_date'] = japan_fri['Date']
japan_wkd['fri_open'] = japan_fri['Open']
japan_wkd['fri_close'] = japan_fri['Close']
japan_wkd['fri_adj_close'] = japan_fri['Adj Close']
japan_wkd['fri_low'] = japan_fri['Low']
japan_wkd['fri_high'] = japan_fri['High']
japan_wkd['fri_volume'] = japan_fri['Volume']
japan_wkd

,ticker,mon_date,mon_open,mon_close,mon_adj_close,mon_low,mon_high,mon_volume,fri_date,fri_open,fri_close,fri_adj_close,fri_low,fri_high,fri_volume
0,1301,2001-01-08,1330.0,1330.0,953.329529,1330.0,1330.0,0.0,2001-01-05,1390.0,1330.0,953.329529,1330.0,1400.0,19700.0
1,1301,2001-01-15,1360.0,1320.0,946.161499,1320.0,1380.0,9000.0,2001-01-12,1320.0,1310.0,938.993591,1310.0,1340.0,59600.0
2,1301,2001-01-22,1370.0,1350.0,967.665161,1330.0,1380.0,10400.0,2001-01-19,1320.0,1320.0,946.161499,1320.0,1350.0,19200.0
3,1301,2001-01-29,1330.0,1420.0,1017.840515,1330.0,1450.0,38400.0,2001-01-26,1330.0,1330.0,953.329529,1310.0,1350.0,34600.0
4,1301,2001-02-05,1420.0,1400.0,1003.504517,1400.0,1430.0,15700.0,2001-02-02,1380.0,1410.0,1010.672485,1380.0,1420.0,16200.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11097,1330,2020-12-28,27530.0,27690.0,27690.000000,27500.0,27700.0,22120.0,2020-12-25,27520.0,27520.0,27520.000000,27490.0,27530.0,38030.0
11098,1330,2021-01-18,29130.0,29180.0,29180.000000,29030.0,29280.0,34530.0,2021-01-15,29810.0,29430.0,29430.000000,29410.0,29830.0,87600.0
11099,1330,2021-01-25,29640.0,29750.0,29750.000000,29500.0,29750.0,19730.0,2021-01-22,29520.0,29560.0,29560.000000,29470.0,29640.0,26380.0
11100,1330,2021-02-01,28550.0,28990.0,28990.000000,28530.0,29020.0,31870.0,2021-01-29,29260.0,28530.0,28530.000000,28530.0,29260.0,81760.0


In [49]:
japan_dst.to_csv('DST_Japan_Stocks.csv')
japan_wkd.to_csv('Weekends_Japan_Stocks.csv')